In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
import getpass
# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [2]:
!pip install cryptocode

In [3]:
import cryptocode

# QKD Protocol

Alice chooses a set of random key string of length 16.

In [5]:
n_bits = 16

Alice_bits = np.random.randint(2,size = n_bits)

Alice's bit string sequence :  [1 0 1 0 1 1 0 1 0 1 1 0 1 1 0 0]


Now, Alice chooses a set of bases {Z,X} and encodes the above random string.

In [6]:
def encode(bits,basis) :
    message = []
    assert len(bits) == len(basis)
    
    for i in range(len(bits)) :
        qc = QuantumCircuit(1,1)
        if basis[i] == "Z" :
            if bits[i] == 0:
                pass
            else :
                qc.x(0)
        else :
            if bits[i] == 0:
                qc.h(0)
            else :
                qc.x(0)
                qc.h(0)
        message.append(qc)
        
    return message

Alice_basis = np.random.choice(["X","Z"],size = n_bits)
print("Alice's bases sequence : ",Alice_basis)

encoded_bitstring = encode(Alice_bits,Alice_basis)
print("Alice's bitstring is successfully encoded now.")

Alice's bases sequence :  ['X' 'Z' 'Z' 'Z' 'Z' 'Z' 'X' 'Z' 'X' 'Z' 'X' 'Z' 'Z' 'Z' 'X' 'Z']
Alice's bitstring is successfully encoded now.


Now, Bob chooses a random set of bases {X,Z} and measures the Alice's bitsring.

In [7]:
Bob_basis = np.random.choice(["X","Z"],size = n_bits)
print("Bob's bases sequence : ",Bob_basis)

def measure(message, basis):
    backend = Aer.get_backend('aer_simulator')
    measurements = []
    for q in range(len(basis)):
        if basis[q] == "Z": 
            message[q].measure(0,0)
        else :
            message[q].h(0)
            message[q].measure(0,0)
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(message[q], shots=1024, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

measured_bitstring = measure(encoded_bitstring,Bob_basis)
print("Encoded bitstring is successfully measured by Bob")

Bob's bases sequence :  ['Z' 'Z' 'X' 'Z' 'Z' 'Z' 'X' 'Z' 'Z' 'X' 'X' 'X' 'Z' 'X' 'X' 'Z']
Encoded bitstring is successfully measured by Bob


Now, Alice and Bob keep the bits corresponding to the bits having same bases.

In [8]:
def common_bits(basis1,basis2,measured) :
    common = []
    for i in range(len(basis1)) :
        if basis1[i] == basis2[i] :
            common.append(measured[i])
    return common

key_bits = common_bits(Alice_basis,Bob_basis,measured_bitstring)

def string_key(k_in) :
    str_key = ""
    for i in range(len(k_in)) :
        str_key += str(k_in[i])
    return str_key

key_string = string_key(key_bits)

print("The private key has now been generated.")

The private key has now been generated.


# Encryption & Decryption

Input from the User: 

In [9]:
def message_in():
    print("Please enter your message")
    message =  getpass.getpass()
    return message

m_in = message_in()

Please enter your message


 ···········


Encrypting message with common key.

In [10]:
key = key_string
str_encoded = cryptocode.encrypt(m_in,key)
print("The message has been Encrypted successfully")

The message has been Encrypted successfully


Decrypting message with common key.

In [11]:
str_decoded = cryptocode.decrypt(str_encoded,key)
print("The message has been Decrypted successfully")

The message has been Decrypted successfully


In [12]:
print("Decoded message = ",str_decoded)

Decoded message =  Hello World
